# RSI model
https://chatgpt.com/c/67f9fbc1-777c-800a-a56a-aadd83458b40

In [17]:
# 📌 Cell 1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# RSI Calculation
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [16]:
# 📌 Cell 2: Load full training data
df = pd.read_csv("XAGUSD-H1-rates.csv", sep=r'\s+', engine='python')
df.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread']

# Convert to datetime
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.sort_values('Datetime')
df.reset_index(drop=True, inplace=True)

# Compute RSI
df['RSI'] = compute_rsi(df['Close'], period=14)

# Drop NaN rows from RSI calculation
df.dropna(inplace=True)


In [18]:
# 📌 Cell 3: Labeling reversal (RSI 30/70)
def label_reversal_targets(data, future_window=5):
    labels = []
    for i in range(len(data) - future_window):
        rsi_now = data.iloc[i]['RSI']
        rsi_future = data.iloc[i+1:i+future_window+1]['RSI']

        if rsi_now < 30 and any(r > 35 for r in rsi_future):
            labels.append(1)  # Bullish reversal
        elif rsi_now > 70 and any(r < 65 for r in rsi_future):
            labels.append(1)  # Bearish reversal
        else:
            labels.append(0)
    return np.array(labels)

labels = label_reversal_targets(df)


In [19]:
# 📌 Cell 4: Prepare input windows
SEQ_LEN = 30
features = ['Open', 'High', 'Low', 'Close', 'RSI']

X = []
for i in range(len(df) - SEQ_LEN - 5):  # 5 for future reversal window
    window = df.iloc[i:i+SEQ_LEN][features].values
    X.append(window)

X = np.array(X)
y = labels[:len(X)]

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (60441, 30, 5)
y shape: (60441,)


In [20]:
# 📌 Cell 5: Train/Test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

# LSTM model
model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, len(features))),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Training
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


I0000 00:00:1744473144.416425   10918 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2238 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/saeed/repositories/machine-learning/tensorflow-course/start-tensorflow/envs/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,033 (78.25 KB)

 Trainable params: 20,033 (78.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


I0000 00:00:1744473146.548780   11475 cuda_dnn.cc:529] Loaded cuDNN version 90800


1511/1511 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8767 - loss: 0.3731 - val_accuracy: 0.8629 - val_loss: 0.4146
Epoch 2/10
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8933 - loss: 0.2829 - val_accuracy: 0.8916 - val_loss: 0.3691
Epoch 3/10
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9147 - loss: 0.2165 - val_accuracy: 0.8726 - val_loss: 0.2839
Epoch 4/10
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9302 - loss: 0.1618 - val_accuracy: 0.8142 - val_loss: 0.3054
Epoch 5/10
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9358 - loss: 0.1447 - val_accuracy: 0.9084 - val_loss: 0.1947
Epoch 6/10
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9398 - loss: 0.1358 - val_accuracy: 0.8942 - val_loss: 0.2276
Epoch 7/10
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9422 - loss: 0.1271 - val_accuracy: 0.9112 - val_loss: 0.1831
Epoch 8/10
1511/1511 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9471 - loss: 0.1177 - val_accurac

In [21]:
# 📌 Cell 6: Predict on rows1-30.csv
df_input = pd.read_csv("rows1-30.csv", delim_whitespace=True, header=None)
df_input.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread']
df_input['RSI'] = compute_rsi(df_input['Close'], period=14)
df_input.dropna(inplace=True)  # Drop NaNs from RSI

# Build input sample
sample = df_input[['Open', 'High', 'Low', 'Close', 'RSI']].values[-30:]
sample = np.expand_dims(sample, axis=0)  # Shape: (1, 30, 5)

# Predict
prediction = model.predict(sample)[0][0]
print("Reversal probability in next 5 candles:", round(prediction * 100, 2), "%")

if prediction > 0.5:
    print("⚠️ Likely reversal ahead!")
else:
    print("✅ No reversal detected.")


/tmp/ipykernel_10918/2689630546.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_input = pd.read_csv("rows1-30.csv", delim_whitespace=True, header=None)


FileNotFoundError: [Errno 2] No such file or directory: 'rows1-30.csv'